In [2]:
import pandas as pd
import assumptions

var = assumptions.var

In [3]:
room_df = pd.read_csv('building_area/APMRooms.csv')

In [4]:
#Returns 
def extract_buildingData(bd_filepath, building_name):
    room_df = pd.read_csv(bd_filepath)
    room_df = room_df[room_df['Category'].str.match('A - Assignable Area')][['Building', 
                                                                             'Room', 
                                                                             'Area', 
                                                                             'Room Use (S)',
                                                                             'Stations']]

    if building_name != room_df[['Building']]['Building'].unique()[0]:
        print('Error: Building Name of input file does not match desired Building Name')
        print('Building Name from file is {}'.format(room_df[['Building']]))
    searchfor = ['Storage', 'Service', 'Telecomm']
    room_df = room_df[room_df['Room Use (S)'].str.contains('|'.join(searchfor), regex=True) == False]
    return room_df

![](images/ashrae2.png)

![](images/ashrae3.png)

![](images/ashrae.png)

In [5]:
room_df = extract_buildingData('building_area/APMRooms.csv', 'Applied Physics and Mathematics')

In [6]:
room_df.head()

,Building,Room,Area,Room Use (S),Stations
38,Applied Physics and Mathematics,B301,172,560 - Media Production (S),1.0
39,Applied Physics and Mathematics,B301A,335,560 - Media Production (S),0.0
40,Applied Physics and Mathematics,B301B,194,560 - Media Production (S),0.0
41,Applied Physics and Mathematics,B302,335,560 - Media Production (S),1.0
42,Applied Physics and Mathematics,B302A,44,560 - Media Production (S),0.0


In [7]:
def impute_buildingData( ):
    
    occupancyImputed_TF = []
    CFMs = []
    for index, row in room_df.iterrows():
        room_CFM = compute_CFM(row)
        try:
            room_CFM
        except NameError:
            print('Error in row ' + str(index))
            print(Rp)
            print(Pz)
            print(Ra)
            print(Az)
        else:
            CFMs.append(room_CFM)
            occupancyImputed_TF.append(occupancyImputed)
            
    return room_df.assign(CFM = CFMs, occupancyImputed_TF = occupancyImputed_TF)

In [ ]:
def compute_CFM(row):
    r_use = row['Room Use (S)']
    room_area = row['Area']
    Pz = row['Stations']
    occupancyImputed = False
     #If Lecture Classroom or Lecture Hall
        if r_use in ['110 - Classroom (S)', 
                     '270 - Open Laboratory (N)',
                     '270 - Open Lab - Restricted (N)',
                     '340 - Conference Room (S)']:
            #if large enough classify as lecture hall
            if Az > 500:
                CFM, occupancyImputed = reqCFM_calculator('lectureHall', Pz, Az)
           #else lecture classroom
            else:
                CFM, occupancyImputed = reqCFM_calculator('lectureClassroom', Pz, Az) 
        #If Office space
        elif r_use in ['211 - Research Office - Enclosed (S)',
                     '335 - Office Service (S)',
                     '320 - Administrative Office - Enclosed (S)', 
                     '211 - Research Office - Open (S)',
                     '310 - Academic Office - Enclosed (S)',
                     '226 - Research Office Service (S)',
                     '320 - Administrative Office - Open (S)',
                     '340 - Conference Room (S)',
                     '250 - Scholarly Activity (S)']:
            CFM, occupancyImputed = reqCFM_calculator('officeSpace', Pz, Az) 
        #if Research Lab
        elif r_use in ['225 - Research Lab / Studio Service (S)',
                     '210 - Research Laboratory / Studio (S)',
                      '260 - Class Laboratory (S)']:
            CFM, occupancyImputed = reqCFM_calculator('collegeLaboratories', Pz, Az) 
       #if media room 
        elif r_use in ['560 - Media Production (S)']:
            CFM, occupancyImputed = reqCFM_calculator('mediaCenter', Pz, Az) 
        #if Shop
        elif r_use in ['710 - Shop - General and Research (S)']:
            CFM, occupancyImputed = reqCFM_calculator('woodMetalShop', Pz, Az)
        else:
            print('New Classification Required:')
            print(r_use)
            print(row['Room'])
        #Calculate CFM to impute
        return CFM, occupancyImputed

In [17]:
def reqCFM_calculator(rclass, Pz, Az ,var = var):
    """"
    Returns the minimum required airflow in cubic feet per minute (CFM) based on room class,
    occupancy and area, Given ANSI/ASHRAE Standards
    url: https://www.ashrae.org/File%20Library/Technical%20Resources/Standards%20and%20Guidelines/Standards%20Addenda/62.1-2016/62_1_2016_s_20190726.pdf
    Formula: Ventilation = Rp * Pz + Ra * Az
    Az == Floor area in sq ft
    Pz == Zone occupancy or zone population 
    Rp == Outdoor Airflow rate in cfm / person 
    Ra == Outdoor Airflow rate in cfm /sq ft  
    Od == Occupant Density. occ/1000 sq ft"""
    af_standards = var['ASHRAE_table']
    occupancyImputed = False
    if Pz == 0:
        Pz = Az / Od_CONVERSION * af_standards[rclass]['Od']
        occupancyImputed = True
    Rp = af_standards[rclass]['Rp']
    Ra = af_standards[rclass]['Ra']
    #
    CFM = Rp * Pz + Ra * Az
    return CFM, occupancyImputed

In [12]:
imputed = impute_buildingData(room_df)
imputed.head()

NameError: name 'compute_CFM' is not defined

In [13]:
CFM_calculator?